In [1]:
!python -m pip install -q paddlepaddle-gpu==2.5.0.post118 -f https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html
!pip install -q "paddleocr>=2.0.1" # Recommend to use version 2.0.1+
!pip install -q imutils
!pip install -q gdown

In [2]:
import paddle
gpu_available  = paddle.device.is_compiled_with_cuda()
print("GPU available:", gpu_available)

GPU available: True


In [3]:
!git clone https://github.com/PaddlePaddle/PaddleOCR.git

Cloning into 'PaddleOCR'...
remote: Enumerating objects: 141189, done.
remote: Counting objects: 100% (9287/9287), done.
remote: Compressing objects: 100% (1048/1048), done.
remote: Total 141189 (delta 8956), reused 8239 (delta 8239), pack-reused 131902 (from 3)
Receiving objects: 100% (141189/141189), 833.30 MiB | 35.83 MiB/s, done.
Resolving deltas: 100% (110469/110469), done.


In [4]:
!cd /workspace/HVDS_DEV/accounts/chidtl/PaddleOCR
# Download the pre-trained model of en_PP-OCRv3
!wget -P ./pretrain_models/ https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_rec_train.tar

/bin/bash: line 0: cd: /workspace/HVDS_DEV/accounts/chidtl/PaddleOCR: No such file or directory
--2025-04-21 14:18:28--  https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_rec_train.tar
Resolving paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)... 103.235.47.176, 2402:2b40:7000:628:0:ff:b0e8:88da
Connecting to paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)|103.235.47.176|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 204093440 (195M) [application/x-tar]
Saving to: './pretrain_models/en_PP-OCRv3_rec_train.tar'

en_PP-OCRv3_rec_tra 100%[===================>] 194.64M  16.6MB/s    in 30s     

2025-04-21 14:18:59 (6.55 MB/s) - './pretrain_models/en_PP-OCRv3_rec_train.tar' saved [204093440/204093440]



In [5]:
!cd /kaggle/working/pretrain_models
!tar -xf /kaggle/working/pretrain_models/en_PP-OCRv3_rec_train.tar -C /kaggle/working/pretrain_models/

In [6]:
!cd PaddleOCR/

In [7]:
!cd /kaggle/working/PaddleOCR

In [11]:
!ls

PaddleOCR  pretrain_models  rec_gt_train_fixed.txt  rec_gt_val_fixed.txt


In [8]:
TXT = '''
Global:
  debug: false
  use_gpu: true
  epoch_num: 40
  log_smooth_window: 20
  print_batch_step: 10
  save_model_dir: ./PaddleOCR/output/v3_en_mobile
  save_epoch_step: 10
  eval_batch_step: [0, 500]
  cal_metric_during_train: true
  pretrained_model:
  checkpoints:
  save_inference_dir:
  use_visualdl: false
  infer_img: doc/imgs_words/ch/word_1.jpg
  character_dict_path: /kaggle/working/PaddleOCR/ppocr/utils/en_dict.txt
  max_text_length: &max_text_length 25
  infer_mode: false
  use_space_char: true
  distributed: true
  save_res_path: ./output/rec/predicts_ppocrv3_en.txt


Optimizer:
  name: Adam
  beta1: 0.9
  beta2: 0.999
  lr:
    name: Cosine
    learning_rate: 0.001
    warmup_epoch: 3
  regularizer:
    name: L2
    factor: 3.0e-05


Architecture:
  model_type: rec
  algorithm: SVTR_LCNet
  Transform:
  Backbone:
    name: MobileNetV1Enhance
    scale: 0.5
    last_conv_stride: [1, 2]
    last_pool_type: avg
    last_pool_kernel_size: [2, 2]
  Head:
    name: MultiHead
    head_list:
      - CTCHead:
          Neck:
            name: svtr
            dims: 64
            depth: 2
            hidden_dims: 120
            use_guide: True
          Head:
            fc_decay: 0.00001
      - SARHead:
          enc_dim: 512
          max_text_length: *max_text_length

Loss:
  name: MultiLoss
  loss_config_list:
    - CTCLoss:
    - SARLoss:

PostProcess:  
  name: CTCLabelDecode

Metric:
  name: RecMetric
  main_indicator: acc
  ignore_space: False

Train:
  dataset:
    name: SimpleDataSet
    data_dir: /kaggle/input/rec-2104/rec/augmented_images_train
    ext_op_transform_idx: 1
    label_file_list:
    - /kaggle/input/rec-2104/rec/augmented_labels_train.txt
    transforms:
    - DecodeImage:
        img_mode: BGR
        channel_first: false
    - RecConAug:
        prob: 0.5
        ext_data_num: 2
        image_shape: [48, 320, 3]
        max_text_length: *max_text_length
    - RecAug:
    - MultiLabelEncode:
    - RecResizeImg:
        image_shape: [3, 48, 320]
    - KeepKeys:
        keep_keys:
        - image
        - label_ctc
        - label_sar
        - length
        - valid_ratio
  loader:
    shuffle: true
    batch_size_per_card: 32
    drop_last: true
    num_workers: 4
Eval:
  dataset:
    name: SimpleDataSet
    data_dir: /kaggle/input/rec-2104/rec/val_set
    label_file_list:
    -  /kaggle/input/rec-2104/rec/rec_gt_val.txt
    transforms:
    - DecodeImage:
        img_mode: BGR
        channel_first: false
    - MultiLabelEncode:
    - RecResizeImg:
        image_shape: [3, 48, 320]
    - KeepKeys:
        keep_keys:
        - image
        - label_ctc
        - label_sar
        - length
        - valid_ratio
  loader:
    shuffle: false
    drop_last: false
    batch_size_per_card: 32
    num_workers: 2
'''
import os
config_path = '/kaggle/working/PaddleOCR/configs/rec/PP-OCRv3/en_PP-OCRv3_rec.yml'
os.makedirs(os.path.dirname(config_path), exist_ok=True)

# ✨ Ghi đè luôn file YAML
with open(config_path, 'w') as f:
    f.write(TXT)

print(f"✅ Đã ghi đè file cấu hình YAML tại: {config_path}")


✅ Đã ghi đè file cấu hình YAML tại: /kaggle/working/PaddleOCR/configs/rec/PP-OCRv3/en_PP-OCRv3_rec.yml


In [9]:
!pwd

/kaggle/working


In [10]:

!python ./PaddleOCR/tools/train.py -c ./PaddleOCR/configs/rec/PP-OCRv3/en_PP-OCRv3_rec.yml -o Global.pretrained_model="/kaggle/working/pretrain_models/en_PP-OCRv3_rec_train/best_accuracy"

[2025/04/21 14:19:09] ppocr WARNING: Skipping import of the encryption module.
[2025/04/21 14:19:09] ppocr INFO: Architecture : 
[2025/04/21 14:19:09] ppocr INFO:     Backbone : 
[2025/04/21 14:19:09] ppocr INFO:         last_conv_stride : [1, 2]
[2025/04/21 14:19:09] ppocr INFO:         last_pool_kernel_size : [2, 2]
[2025/04/21 14:19:09] ppocr INFO:         last_pool_type : avg
[2025/04/21 14:19:09] ppocr INFO:         name : MobileNetV1Enhance
[2025/04/21 14:19:09] ppocr INFO:         scale : 0.5
[2025/04/21 14:19:09] ppocr INFO:     Head : 
[2025/04/21 14:19:09] ppocr INFO:         head_list : 
[2025/04/21 14:19:09] ppocr INFO:             CTCHead : 
[2025/04/21 14:19:09] ppocr INFO:                 Head : 
[2025/04/21 14:19:09] ppocr INFO:                     fc_decay : 1e-05
[2025/04/21 14:19:09] ppocr INFO:                 Neck : 
[2025/04/21 14:19:09] ppocr INFO:                     depth : 2
[2025/04/21 14:19:09] ppocr INFO:                     dims : 64
[2025/04/21 14:19:09] 

In [11]:
!python /kaggle/working/PaddleOCR/tools/export_model.py -c /kaggle/working/PaddleOCR/configs/rec/PP-OCRv3/en_PP-OCRv3_rec.yml -o Global.pretrained_model="./PaddleOCR/output/v3_en_mobile/latest"  Global.save_inference_dir="./PaddleOCR/inference/en_PP-OCRv3_rec/"

[2025/04/21 14:37:33] ppocr WARNING: Skipping import of the encryption module.
W0421 14:37:34.548893  1010 gpu_resources.cc:119] Please NOTE: device: 0, GPU Compute Capability: 7.5, Driver API Version: 12.6, Runtime API Version: 11.8
W0421 14:37:34.550278  1010 gpu_resources.cc:149] device: 0, cuDNN Version: 8.9.
[2025/04/21 14:37:34] ppocr INFO: load pretrain successful from ./PaddleOCR/output/v3_en_mobile/latest
[2025/04/21 14:37:34] ppocr INFO: Export inference config file to ./PaddleOCR/inference/en_PP-OCRv3_rec/inference.yml
Skipping import of the encryption module
I0421 14:37:36.505053  1010 interpretercore.cc:237] New Executor is Running.
[2025/04/21 14:37:36] ppocr INFO: inference model is saved to ./PaddleOCR/inference/en_PP-OCRv3_rec/inference


In [13]:
!python /kaggle/working/PaddleOCR/tools/infer_rec.py -c /kaggle/working/PaddleOCR/configs/rec/PP-OCRv3/en_PP-OCRv3_rec.yml -o Global.pretrained_model="./PaddleOCR/output/v3_en_mobile/latest"  Global.infer_img="/kaggle/input/rec-2104/rec/val_set/10xemay2192_0.jpg"

[2025/04/21 14:38:09] ppocr WARNING: Skipping import of the encryption module.
[2025/04/21 14:38:09] ppocr INFO: Architecture : 
[2025/04/21 14:38:09] ppocr INFO:     Backbone : 
[2025/04/21 14:38:09] ppocr INFO:         last_conv_stride : [1, 2]
[2025/04/21 14:38:09] ppocr INFO:         last_pool_kernel_size : [2, 2]
[2025/04/21 14:38:09] ppocr INFO:         last_pool_type : avg
[2025/04/21 14:38:09] ppocr INFO:         name : MobileNetV1Enhance
[2025/04/21 14:38:09] ppocr INFO:         scale : 0.5
[2025/04/21 14:38:09] ppocr INFO:     Head : 
[2025/04/21 14:38:09] ppocr INFO:         head_list : 
[2025/04/21 14:38:09] ppocr INFO:             CTCHead : 
[2025/04/21 14:38:09] ppocr INFO:                 Head : 
[2025/04/21 14:38:09] ppocr INFO:                     fc_decay : 1e-05
[2025/04/21 14:38:09] ppocr INFO:                 Neck : 
[2025/04/21 14:38:09] ppocr INFO:                     depth : 2
[2025/04/21 14:38:09] ppocr INFO:                     dims : 64
[2025/04/21 14:38:09] 

In [17]:
!zip -r /kaggle/working/PaddleOCR/output.zip /kaggle/working/PaddleOCR/output



  adding: kaggle/working/PaddleOCR/output/ (stored 0%)
  adding: kaggle/working/PaddleOCR/output/v3_en_mobile/ (stored 0%)
  adding: kaggle/working/PaddleOCR/output/v3_en_mobile/iter_epoch_20.pdopt (deflated 23%)
  adding: kaggle/working/PaddleOCR/output/v3_en_mobile/iter_epoch_10.states (deflated 9%)
  adding: kaggle/working/PaddleOCR/output/v3_en_mobile/config.yml (deflated 63%)
  adding: kaggle/working/PaddleOCR/output/v3_en_mobile/best_accuracy.pdopt (deflated 23%)
  adding: kaggle/working/PaddleOCR/output/v3_en_mobile/latest.pdopt (deflated 23%)
  adding: kaggle/working/PaddleOCR/output/v3_en_mobile/best_model/ (stored 0%)
  adding: kaggle/working/PaddleOCR/output/v3_en_mobile/best_model/model.pdopt (deflated 23%)
  adding: kaggle/working/PaddleOCR/output/v3_en_mobile/best_model/model.pdparams (deflated 7%)
  adding: kaggle/working/PaddleOCR/output/v3_en_mobile/iter_epoch_30.states (deflated 9%)
  adding: kaggle/working/PaddleOCR/output/v3_en_mobile/iter_epoch_20.pdparams (deflate

In [18]:
!zip -r /kaggle/working/PaddleOCR/inference.zip /kaggle/working/PaddleOCR/inference

  adding: kaggle/working/PaddleOCR/inference/ (stored 0%)
  adding: kaggle/working/PaddleOCR/inference/en_PP-OCRv3_rec/ (stored 0%)
  adding: kaggle/working/PaddleOCR/inference/en_PP-OCRv3_rec/inference.yml (deflated 58%)
  adding: kaggle/working/PaddleOCR/inference/en_PP-OCRv3_rec/inference.pdmodel (deflated 90%)
  adding: kaggle/working/PaddleOCR/inference/en_PP-OCRv3_rec/inference.pdiparams.info (deflated 82%)
  adding: kaggle/working/PaddleOCR/inference/en_PP-OCRv3_rec/inference.pdiparams (deflated 11%)


In [15]:
!zip -r /kaggle/working/PaddleOCR/inference/en_PP-OCRv3_rec.zip /kaggle/working/PaddleOCR/inference/en_PP-OCRv3_rec

  adding: kaggle/working/PaddleOCR/inference/en_PP-OCRv3_rec/ (stored 0%)
  adding: kaggle/working/PaddleOCR/inference/en_PP-OCRv3_rec/inference.pdiparams (deflated 11%)
  adding: kaggle/working/PaddleOCR/inference/en_PP-OCRv3_rec/inference.pdmodel (deflated 90%)
  adding: kaggle/working/PaddleOCR/inference/en_PP-OCRv3_rec/inference.yml (deflated 58%)
  adding: kaggle/working/PaddleOCR/inference/en_PP-OCRv3_rec/inference.pdiparams.info (deflated 81%)
